## Question3: MDP

### This notebook has a vectorized implementation. 
> **After vectorization time reduced from 7minutes/iteration to 10 seconds/iteration if size = (20, 10, 10).**<br/>
> **After vectorization time reduced from 6seconds/iteration to 0.44 second/iteration if size = (10, 5, 5).**

In [2]:
import itertools
import numpy as np
from tqdm import tqdm, tqdm_notebook
import copy

In [2]:
def poisson(lambda1, n):
    return (np.power(lambda1, n)/np.math.factorial(n))*(np.exp(-lambda1))

##########CONSTANTS###########
requestLambda = [3, 2, 2]
returnLambda = [3, 1, 1]

#Number of maximum cars allowed at loc1, loc2, loc3
ml1, ml2, ml3 = 20, 10, 10

gamma = 0.9
numIter = 10
tol = 0.01
Size = (ml1, ml2, ml3)

n1 = np.arange(ml1)
n2 = np.arange(ml2)
n3 = np.arange(ml3)
n4 = np.arange(-5, 6)
#=============================#

#================PreCalculations of Probability and Rewards============#
JointProbLocation1 = np.ones((ml1, ml1))*0
JointProbLocation2 = np.ones((ml2, ml2))*0
JointProbLocation3 = np.ones((ml3, ml3))*0
JointRewardLocation1 = np.ones((ml1, ml1))*0
JointRewardLocation2 = np.ones((ml2, ml2))*0
JointRewardLocation3 = np.ones((ml3, ml3))*0

ProbRequestLocation1 = np.asarray([poisson(requestLambda[0], i) for i in range(ml1)])
ProbRequestLocation2 = np.asarray([poisson(requestLambda[1], i) for i in range(ml2)])
ProbRequestLocation3 = np.asarray([poisson(requestLambda[2], i) for i in range(ml3)])

RewardRequestLocation1 = np.asarray([poisson(requestLambda[0], i)*i*10 for i in range(ml1)])
RewardRequestLocation2 = np.asarray([poisson(requestLambda[1], i)*i*10 for i in range(ml2)])
RewardRequestLocation3 = np.asarray([poisson(requestLambda[2], i)*i*10 for i in range(ml3)])

ProbReturnLocation1 = np.asarray([poisson(returnLambda[0], i) for i in range(ml1)])
ProbReturnLocation2 = np.asarray([poisson(returnLambda[1], i) for i in range(ml2)])
ProbReturnLocation3 = np.asarray([poisson(returnLambda[2], i) for i in range(ml3)])


for (currState, nextState) in itertools.product(n1, n1):  
    for request, ret in itertools.product(n1, n1):
        if(request > currState or currState-request+ret != nextState):
            continue
        JointProbLocation1[currState, nextState] += (ProbRequestLocation1[request]
                            *ProbReturnLocation1[ret])
        JointRewardLocation1[currState, nextState] += (ProbRequestLocation1[request]
                            *ProbReturnLocation1[ret])*request*10
            
for (currState, nextState) in itertools.product(n2, n2):  
    for request, ret in itertools.product(n2, n2):
        if(request > currState or currState-request+ret != nextState):
            continue
        JointProbLocation2[currState, nextState] += (ProbRequestLocation2[request]
                            *ProbReturnLocation2[ret])
        JointRewardLocation2[currState, nextState] += (ProbRequestLocation2[request]
                            *ProbReturnLocation2[ret])*request*10

for (currState, nextState) in itertools.product(n3, n3):  
    for request, ret in itertools.product(n3, n3):
        if(request > currState or currState-request+ret != nextState):
            continue
        JointProbLocation3[currState, nextState] += (ProbRequestLocation3[request]
                            *ProbReturnLocation3[ret])
        JointRewardLocation3[currState, nextState] += (ProbRequestLocation3[request]
                            *ProbReturnLocation3[ret])*request*10
#=========================_________________________========================#
        
def normInfinity(currValue, optimalValue):
    maxDiff = np.max(np.abs(currValue - optimalValue))
    return maxDiff
        

def isValidState(car1, car2, car3):
     return car1 >= 0 and car2 >=0 and car3 >=0 and car1 <ml1 and car2 < ml2 and car3 < ml3

def getCost(mv1, mv2, mv3):
    cost = np.abs(mv1)*2
    return cost

MOVES = []
for mv1, mv2, mv3 in itertools.product(n4, n4, n4):
    if(mv1 + mv2 + mv3 != 0):
        continue
    MOVES.append((mv1, mv2, mv3))


#================================ Value Iteration ===========================#
def ValueIteration(V, Policy):
    for iter1 in (range(numIter)):
        oldV = V.copy()
        print("iter: ", iter1)
        a = time.time()
        #State value
        for i, j, k in itertools.product(n1, n2, n3):
#             print(i, j, k)
            maxReward = -10000.0
            for (mv1, mv2, mv3) in MOVES:
#                 print("move1 = ", mv1, mv2, mv3)
                reward = 0.0
                car1 = i + mv1
                car2 = j + mv2
                car3 = k + mv3
#                 print(i, j, k, car1, car2, car3)
                
                if(not isValidState(car1, car2, car3)):
                    continue
                
                reward-=getCost(mv1, mv2, mv3)
                
                p1_sum = JointProbLocation1[car1, :]   #(10, )
                p2_sum = JointProbLocation2[car2, :]   #(5, )
                p3_sum = JointProbLocation3[car3, :]   #(5, )
                r1_sum = JointRewardLocation1[car1, :].reshape(ml1, 1)  #(10, 1)
                r2_sum = JointRewardLocation2[car2, :].reshape(ml2, 1)   #(5, 1)
                r3_sum = JointRewardLocation3[car3, :].reshape(ml3, 1)    #(5, 1)
                
                p1p2_sum = np.dot(p1_sum.reshape(ml1, 1), p2_sum.reshape(1, ml2))  #(ml1, ml2)
                p1p3_sum = np.dot(p1_sum.reshape(ml1, 1), p3_sum.reshape(1, ml3))  #(ml1, ml3)
                p2p3_sum = np.dot(p2_sum.reshape(ml2, 1), p3_sum.reshape(1, ml3))  #(ml2, ml3)
        
                #========immediate Reward ======================#
                r3p1p2 = np.sum(np.dot(np.tile(r3_sum, (1, ml1)), p1p2_sum))
                r2p1p3 = np.sum(np.dot(np.tile(r2_sum, (1, ml1)), p1p3_sum))
                r1p2p3 = np.sum(np.dot(np.tile(r1_sum, (1, ml2)), p2p3_sum))
                
                immediateReward = r3p1p2 + r2p1p3 + r1p2p3
                #==================================================#
                
                #============Value from next State=================#
                temp = np.tile(p2p3_sum, (ml1, 1, 1))
                futureReward = np.multiply(temp, oldV)
                temp1 = np.tile(p1_sum.reshape(ml1, 1, 1), (1, ml2, ml3))
                futureReward = np.sum(np.multiply(futureReward, temp1))
                #==================================================#
                
                reward += immediateReward+ gamma*futureReward
                if(reward > maxReward):
                    maxReward = reward
                    Policy[i, j, k] = np.array([mv1, mv2, mv3])
            V[i, j, k] = maxReward 

        if(normInfinity(V, oldV) <= 0.1):
            print("Converged in {} iteration".format(iter1 + 2))
            break
        print("Time for 1 iter = ",time.time() - a)
    return V, Policy

In [ ]:
import time
Size = (ml1, ml2, ml3)
Value = np.zeros(Size)
Policy = np.zeros(Size + tuple([3]))
a = time.time()
V, P = ValueIteration(Value, Policy)
print("Time Taken = ",time.time() - a)
np.savetxt("Value2.txt", np.ravel(V).reshape(ml1*ml2, ml3), fmt = "%.2f", header = 'values')
np.savetxt("Policy1.txt", np.ravel(P).reshape(ml1*ml2*ml3, 3), fmt = "%i")
print("Value2.txt Saved")
print("Policy2.txt Saved")

iter:  0
Time for 1 iter =  3350.737420797348
iter:  1


### Vectorization Improvement

In [1]:
def poisson(lambda1, n):
    return (np.power(lambda1, n)/np.math.factorial(n))*(np.exp(-lambda1))

##########CONSTANTS###########
requestLambda = [3, 2, 2]
returnLambda = [3, 1, 1]

#Number of maximum cars allowed at loc1, loc2, loc3
ml1, ml2, ml3 = 20, 10, 10

gamma = 0.9
numIter = 10
tol = 0.01
Size = (ml1, ml2, ml3)

n1 = np.arange(ml1)
n2 = np.arange(ml2)
n3 = np.arange(ml3)
n4 = np.arange(-5, 6)
#=============================#

#================PreCalculations of Probability and Rewards============#
JointProbLocation1 = np.ones((ml1, ml1))*0
JointProbLocation2 = np.ones((ml2, ml2))*0
JointProbLocation3 = np.ones((ml3, ml3))*0
JointRewardLocation1 = np.ones((ml1, ml1))*0
JointRewardLocation2 = np.ones((ml2, ml2))*0
JointRewardLocation3 = np.ones((ml3, ml3))*0

ProbRequestLocation1 = np.asarray([poisson(requestLambda[0], i) for i in range(ml1)])
ProbRequestLocation2 = np.asarray([poisson(requestLambda[1], i) for i in range(ml2)])
ProbRequestLocation3 = np.asarray([poisson(requestLambda[2], i) for i in range(ml3)])

RewardRequestLocation1 = np.asarray([poisson(requestLambda[0], i)*i*10 for i in range(ml1)])
RewardRequestLocation2 = np.asarray([poisson(requestLambda[1], i)*i*10 for i in range(ml2)])
RewardRequestLocation3 = np.asarray([poisson(requestLambda[2], i)*i*10 for i in range(ml3)])

ProbReturnLocation1 = np.asarray([poisson(returnLambda[0], i) for i in range(ml1)])
ProbReturnLocation2 = np.asarray([poisson(returnLambda[1], i) for i in range(ml2)])
ProbReturnLocation3 = np.asarray([poisson(returnLambda[2], i) for i in range(ml3)])


for (currState, nextState) in itertools.product(n1, n1):  
    for request, ret in itertools.product(n1, n1):
        if(request > currState or currState-request+ret != nextState):
            continue
        JointProbLocation1[currState, nextState] += (ProbRequestLocation1[request]
                            *ProbReturnLocation1[ret])
        JointRewardLocation1[currState, nextState] += (ProbRequestLocation1[request]
                            *ProbReturnLocation1[ret])*request*10
            
for (currState, nextState) in itertools.product(n2, n2):  
    for request, ret in itertools.product(n2, n2):
        if(request > currState or currState-request+ret != nextState):
            continue
        JointProbLocation2[currState, nextState] += (ProbRequestLocation2[request]
                            *ProbReturnLocation2[ret])
        JointRewardLocation2[currState, nextState] += (ProbRequestLocation2[request]
                            *ProbReturnLocation2[ret])*request*10

for (currState, nextState) in itertools.product(n3, n3):  
    for request, ret in itertools.product(n3, n3):
        if(request > currState or currState-request+ret != nextState):
            continue
        JointProbLocation3[currState, nextState] += (ProbRequestLocation3[request]
                            *ProbReturnLocation3[ret])
        JointRewardLocation3[currState, nextState] += (ProbRequestLocation3[request]
                            *ProbReturnLocation3[ret])*request*10
#=========================_________________________========================#
        
def normInfinity(currValue, optimalValue):
    maxDiff = np.max(np.abs(currValue - optimalValue))
    return maxDiff
        

def isValidState(car1, car2, car3):
     return car1 >= 0 and car2 >=0 and car3 >=0 and car1 <ml1 and car2 < ml2 and car3 < ml3

def getCost(mv1, mv2, mv3):
    cost = np.abs(mv1)*2
    return cost

MOVES = []
for mv1, mv2, mv3 in itertools.product(n4, n4, n4):
    if(mv1 + mv2 + mv3 != 0):
        continue
    MOVES.append((mv1, mv2, mv3))


#================================ Value Iteration ===========================#
def ValueIteration(V, Policy):
    for iter1 in (range(numIter)):
        oldV = V.copy()
        print("iter: ", iter1)
        a = time.time()
        #State value
        for i, j, k in itertools.product(n1, n2, n3):
#             print(i, j, k)
            maxReward = -10000.0
            for (mv1, mv2, mv3) in MOVES:
#                 print("move1 = ", mv1, mv2, mv3)
                reward = 0.0
                car1 = i + mv1
                car2 = j + mv2
                car3 = k + mv3
#                 print(i, j, k, car1, car2, car3)
                
                if(not isValidState(car1, car2, car3)):
                    continue
                
                reward-=getCost(mv1, mv2, mv3)
                
                p1_sum = JointProbLocation1[car1, :]   #(10, )
                p2_sum = JointProbLocation2[car2, :]   #(5, )
                p3_sum = JointProbLocation3[car3, :]   #(5, )
                r1_sum = JointRewardLocation1[car1, :].reshape(ml1, 1)  #(10, 1)
                r2_sum = JointRewardLocation2[car2, :].reshape(ml2, 1)   #(5, 1)
                r3_sum = JointRewardLocation3[car3, :].reshape(ml3, 1)    #(5, 1)
                
                p1p2_sum = np.dot(p1_sum.reshape(ml1, 1), p2_sum.reshape(1, ml2))  #(ml1, ml2)
                p1p3_sum = np.dot(p1_sum.reshape(ml1, 1), p3_sum.reshape(1, ml3))  #(ml1, ml3)
                p2p3_sum = np.dot(p2_sum.reshape(ml2, 1), p3_sum.reshape(1, ml3))  #(ml2, ml3)
        
                #========immediate Reward ======================#
                r3p1p2 = np.sum(np.dot(np.tile(r3_sum, (1, ml1)), p1p2_sum))
                r2p1p3 = np.sum(np.dot(np.tile(r2_sum, (1, ml1)), p1p3_sum))
                r1p2p3 = np.sum(np.dot(np.tile(r1_sum, (1, ml2)), p2p3_sum))
                
                immediateReward = r3p1p2 + r2p1p3 + r1p2p3
                #==================================================#
                
                #============Value from next State=================#
                temp = np.tile(p2p3_sum, (ml1, 1, 1))
                futureReward = np.multiply(temp, oldV)
                temp1 = np.tile(p1_sum.reshape(ml1, 1, 1), (1, ml2, ml3))
                futureReward = np.sum(np.multiply(futureReward, temp1))
                #==================================================#
                
                reward += immediateReward+ gamma*futureReward
                if(reward > maxReward):
                    maxReward = reward
                    Policy[i, j, k] = np.array([mv1, mv2, mv3])
            V[i, j, k] = maxReward 

        if(normInfinity(V, oldV) <= 0.1):
            print("Converged in {} iteration".format(iter1 + 2))
            break
        print("Time for 1 iter = ",time.time() - a)
    return V, Policy

NameError: name 'np' is not defined

In [ ]:
import time
Size = (ml1, ml2, ml3)
Value = np.zeros(Size)
Policy = np.zeros(Size + tuple([3]))
a = time.time()
V, P = ValueIteration(Value, Policy)
print("Time Taken = ",time.time() - a)
np.savetxt("Value2.txt", np.ravel(V).reshape(ml1*ml2, ml3), fmt = "%.2f", header = 'values')
np.savetxt("Policy1.txt", np.ravel(P).reshape(ml1*ml2*ml3, 3), fmt = "%i")
print("Value2.txt Saved")
print("Policy2.txt Saved")

---